# Hindi Character Recognition

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [2]:
import cv2
from keras.models import load_model
import numpy as np
import os
from collections import deque

### Loading trained Model

In [3]:
model = load_model('Hindi_charcter_recognition.h5')

### Loading labels

In [4]:
# Dictionary for labels
labels = {}

In [5]:
# Path to classes
path = 'Dataset/Test'

In [6]:
# Adding lables in Dictionary
i = 0
for folder in os.listdir(path):
    fold = folder[13:]
    labels[i] = fold
    i += 1

In [7]:
labels

{0: 'na',
 1: 'adna',
 2: 'gya',
 3: 'yaw',
 4: 'pha',
 5: 'ma',
 6: 'ja',
 7: 'gha',
 8: 'tra',
 9: 'dhaa',
 10: 'ka',
 11: 'waw',
 12: 'bha',
 13: 'thaa',
 14: 'daa',
 15: 'tabala',
 16: 'kha',
 17: 'kna',
 18: 'ga',
 19: 'pa',
 20: 'ra',
 21: 'yna',
 22: 'patalosaw',
 23: 'dha',
 24: 'cha',
 25: 'taamatar',
 26: 'petchiryakha',
 27: 'tha',
 28: 'da',
 29: 'motosaw',
 30: 'ha',
 31: 'chhya',
 32: 'chha',
 33: 'ba',
 34: 'la',
 35: 'jha'}

### Initialising Camera and ROI

In [8]:
# Opening Camera
cap = cv2.VideoCapture(0)

In [9]:
pred_class=0
pts = deque(maxlen=512)
blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
digit = np.zeros((200, 200, 3), dtype=np.uint8)
flag = True
while flag:
    # Read image
    ret, frame = cap.read()
    # Flip Image
    frame = cv2.flip(frame, 1)
    # BGR2HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Extracting color from image
    mask = cv2.inRange(hsv, np.array([110, 50, 50]), np.array([130, 255, 255]))
    # Blurring image
    median = cv2.medianBlur(mask, 15)
    blur = cv2.GaussianBlur(median, (5,5), 0)
    
    # Threshold image
    ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Finding Contours
    contours, heirarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    center = None
    
    if len(contours) >= 1:
        contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(contour) > 250:
            ((x, y), radius) = cv2.minEnclosingCircle(contour)
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
            M = cv2.moments(contour)
            center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
            pts.appendleft(center)
            for i in range(1, len(pts)):
                if pts[i - 1] is None or pts[i] is None:
                    continue
                cv2.line(blackboard, pts[i - 1], pts[i], (255, 255, 255), 10)
                cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), 5)
    
    elif len(contours) == 0:
        if len(pts) != []:
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur1 = cv2.medianBlur(blackboard_gray, 15)
            blur1 = cv2.GaussianBlur(blur1, (5, 5), 0)
            thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            blackboard_cnts, heirarchy_1 = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            if len(blackboard_cnts) >= 1:
                cnt = max(blackboard_cnts, key=cv2.contourArea)
                if cv2.contourArea(cnt) > 2000:
                    x, y, w, h = cv2.boundingRect(cnt)
                    digit = blackboard_gray[y:y + h, x:x + w]
                    # newImage = process_letter(digit)
                    
                    im = cv2.resize(digit, (32, 32))
                    im = np.array(im, dtype=np.float32)
                    im = np.reshape(im, (-1, 32, 32, 1))
                    pred_probab = model.predict(im)[0]
                    pred_class = list(pred_probab).index(max(pred_probab))

            pts = deque(maxlen=512)
            blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
    
    cv2.putText(frame, "Conv Network :  " + str(labels[pred_class]), (10, 470),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # Showing predictions
    cv2.imshow('asd',frame)
    cv2.imshow("Contours", thresh)
      
    # Taking input key
    keypress = cv2.waitKey(1)
        
    # If "q" is pressed
    # Terminate
    if keypress == ord('q'):
        flag = False

cap.release()
cv2.destroyAllWindows()